<a href="https://colab.research.google.com/github/Snazz2001/100-Days-Of-ML-Code/blob/master/RAG_chatgroq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-core langchain-community pypdf pymupdf faiss-cpu sentence-transformers  langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requ

In [20]:
!pip install langchain-text-splitters langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.4/477.4 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1


In [2]:
import os, textwrap, gdown, torch
from google.colab import userdata
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

In [3]:
key = userdata.get('GROQ_API_KEY')
if key:
    os.environ['GROQ_API_KEY'] = key

In [4]:
directory_name = "pdf_files"
os.makedirs(directory_name, exist_ok=True)
print(f"Directory '{directory_name}' is ready.")

Directory 'pdf_files' is ready.


In [5]:
links = [
    # You can replace with yours
     "https://drive.google.com/file/d/17ujrBMddGf_kOx4yjUKbPa0gUJt8POZT/view?usp=sharing",  # file
     "https://drive.google.com/file/d/1SYi854OUQoCOWJ2mV-GKDndI8sGTAP4p/view?usp=sharing",  # file
]

In [6]:
for i, url in enumerate(links, start=1):
    output_path = os.path.join(directory_name, f"file_{i}.pdf")  # saves as file_1.pdf, file_2.pdf...
    gdown.download(url=url, output=output_path, quiet=False, fuzzy=True)
    print(f"Downloaded: {output_path}")

print("✅ All PDF files downloaded successfully!")

Downloading...
From: https://drive.google.com/uc?id=17ujrBMddGf_kOx4yjUKbPa0gUJt8POZT
To: /content/pdf_files/file_1.pdf
100%|██████████| 2.27M/2.27M [00:00<00:00, 172MB/s]


Downloaded: pdf_files/file_1.pdf


Downloading...
From: https://drive.google.com/uc?id=1SYi854OUQoCOWJ2mV-GKDndI8sGTAP4p
To: /content/pdf_files/file_2.pdf
100%|██████████| 1.35M/1.35M [00:00<00:00, 108MB/s]

Downloaded: pdf_files/file_2.pdf
✅ All PDF files downloaded successfully!


In [7]:
def process_all_pdfs(pdf_directory):
    """Load and extract all PDFs in a directory using LangChain's PyPDFLoader"""
    pdf_dir = Path(pdf_directory)
    pdf_files = list(pdf_dir.rglob("*.pdf"))

    print(f"Found {len(pdf_files)} PDF file(s).")
    all_documents = []

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()  # one Document per page
            #metadata
            for doc in documents:
                doc.metadata.update({
                    "source_file": pdf_file.name,
                    "file_type": "pdf"
                })
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} page(s)")
        except Exception as e:
            print(f"  ✗ Error reading {pdf_file.name}: {e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# PDF Files Directory
all_pdf_documents = process_all_pdfs("/content/pdf_files")

Found 2 PDF file(s).

Processing: file_2.pdf
  ✓ Loaded 21 page(s)

Processing: file_1.pdf
  ✓ Loaded 15 page(s)

Total documents loaded: 36


In [8]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-31T00:20:38+00:00', 'author': '', 'keywords': '', 'moddate': '2024-10-31T00:20:38+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/pdf_files/file_2.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1', 'source_file': 'file_2.pdf', 'file_type': 'pdf'}, page_content='YOLOv10: Real-Time End-to-End Object Detection\nAo Wang1 Hui Chen2∗ Lihao Liu1 Kai Chen1\nZijia Lin1 Jungong Han3 Guiguang Ding1∗\n1School of Software, Tsinghua University 2BNRist, Tsinghua University\n3Department of Automation, Tsinghua University\nwa22@mails.tsinghua.edu.cn huichen@mail.tsinghua.edu.cn linzijia07@tsinghua.org.cn\n{louisliu2048,chenkai2010.9,jungonghan77}@gmail.com dinggg@tsinghua.edu.cn\n/uni00000015/uni00000011/uni00000018/uni00000018/uni00000011/uni00000013/uni0

In [9]:
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for RAG."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = splitter.split_documents(documents)
    print(f"📄 {len(documents)} docs → 🔹 {len(chunks)} chunks")
    return chunks


In [10]:
chunks=split_documents(all_pdf_documents)
chunks

📄 36 docs → 🔹 194 chunks


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-31T00:20:38+00:00', 'author': '', 'keywords': '', 'moddate': '2024-10-31T00:20:38+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/pdf_files/file_2.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1', 'source_file': 'file_2.pdf', 'file_type': 'pdf'}, page_content='YOLOv10: Real-Time End-to-End Object Detection\nAo Wang1 Hui Chen2∗ Lihao Liu1 Kai Chen1\nZijia Lin1 Jungong Han3 Guiguang Ding1∗\n1School of Software, Tsinghua University 2BNRist, Tsinghua University\n3Department of Automation, Tsinghua University\nwa22@mails.tsinghua.edu.cn huichen@mail.tsinghua.edu.cn linzijia07@tsinghua.org.cn\n{louisliu2048,chenkai2010.9,jungonghan77}@gmail.com dinggg@tsinghua.edu.cn\n/uni00000015/uni00000011/uni00000018/uni00000018/uni00000011/uni00000013/uni0

In [11]:
# Pick GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Embedding model
emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},
)

# Build the FAISS vector store from your `chunks`
vs = FAISS.from_documents(chunks, emb)

# Save the index to disk
index_dir = "faiss_index"
vs.save_local(index_dir)
print(f"✅ Saved FAISS index to: {index_dir}")

Using device: cpu


/tmp/ipython-input-2769724696.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(


✅ Saved FAISS index to: faiss_index


In [12]:
# Reload
vs = FAISS.load_local("faiss_index", emb, allow_dangerous_deserialization=True)

# Query the vector DB
query = "What are the key contributions of YOLOv10?"
results = vs.similarity_search(query, k=4)  # returns top-4 chunks

for i, doc in enumerate(results, 1):
    print(f"\n[{i}] {doc.metadata.get('source_file')} • page {doc.metadata.get('page')}")
    print(doc.page_content[:400], "...")


[1] file_2.pdf • page 20
medical image analyses and autonomous driving, etc. We hope that our YOLOv10 can assist in these
fields and improve the efficiency. However, we acknowledge the potential for malicious use of our
models. We will make every effort to prevent this.
21 ...

[2] file_2.pdf • page 6
YOLOv10 models. YOLOv10 has the same variants as YOLOv8, i.e., N / S / M / L / X. Besides, we
derive a new variant YOLOv10-B, by simply increasing the width scale factor of YOLOv10-M. We
verify the proposed detector on COCO [35] under the same training-from-scratch setting [21, 65, 62].
Moreover, the latencies of all models are tested on T4 GPU with TensorRT FP16, following [78].
4.2 Comparison wi ...

[3] file_2.pdf • page 6
YOLOv10-L (Ours) 24.4 120.3 53.2 / 53.4† 7.28 7.21
YOLOv8-X [21] 68.2 257.8 53.9 16.86 12.83
RT-DETR-R101 [78] 76.0 259.0 54.3 13.71 13.58
YOLOv10-X (Ours) 29.5 160.4 54.4 / 54.4† 10.70 10.60
quadratic computational complexity of self-attention. In this way, the glo

In [13]:
api_key =  userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(temperature = 0.0, model= "gpt-3.5-turbo", openai_api_key=api_key)

In [18]:
# llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",temperature=0.1,max_tokens=1024)

In [15]:
def rag_answer(query, vs, llm, k=4, query_prefix="", max_chars=30000, per_doc_limit=6000):
    """
    llm: gemma2-9b-it
    """
    # 1) Retrieve
    docs = vs.similarity_search(query_prefix + query, k=k)

    # 2) Build compact context
    context = "\n\n---\n\n".join(
        (d.page_content or "").strip() for d in docs if d.page_content
    )

    # 3) Ask Groq LLM
    prompt = textwrap.dedent(f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer
    """).strip()

    resp = llm.invoke(prompt)
    return (getattr(resp, "content", None) or str(resp)).strip()

# Question
q = "Provide me an comparison between YOLOv7 and YOLOv10"
print(rag_answer(q, vs, llm, k=6))

YOLOv7 has 75% less parameters, 36% less computation, and brings 1.5% higher AP compared to YOLOv4. In contrast, YOLOv10 achieves 1.2% to 1.4% AP improvements with 28% to 57% fewer parameters, 23% to 38% less calculations, and 37% to 70% lower latencies compared to YOLOv8.
